In [16]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

In [17]:
avg_hourly_rain_file = '12_avg.csv'
all_rain_gages_data =  '2018-12-29_2018-12-30GSSHA_rain_gages.csv'

# Open Up Average mm/hr Rain File from ArcGIS calculation

In [18]:
#read raster values
rain_meta = pd.read_csv(avg_hourly_rain_file)
avg_rain = rain_meta.filter(['FILENAME','RASTERVALU'], axis=1)

#read gage data
gages_data = pd.read_csv(all_rain_gages_data)

In [19]:
avg_rain

FILENAME  RASTERVALU
0    911620-22501    0.097225
1    911650-22536    0.161646
2    911700-22508    0.196284
3    911760-22519    0.147491
4    911780-22551    0.096568
..            ...         ...
288         WMLH1    0.202048
289         WMTH1    0.119401
290         WPPH1    0.324691
291         WUHH1    0.250420
292         WUKH1    0.179793

[293 rows x 2 columns]

# Determine which rain gauges have missing values

In [20]:
#get the names of gages from columns
gages_names = (gages_data.columns.tolist())[2:]
n = len(gages_data)
#Find out which columns have missing values... 
for i in range(0, len(gages_names)):
    name = gages_names[i]
    na_val = gages_data[name].isna().sum()
    print(name +' = '+ str(na_val) + ' NaN')
    
print('length of rain data: ' + str(n))

HI-41 = 0 NaN
HI-45 = 0 NaN
KPIH1 = 0 NaN
PRIH1 = 0 NaN
USC00514561 = 25 NaN
USC00518165 = 0 NaN
USGS_uv220356159281401 = 0 NaN
USGS_uv220427159300201 = 0 NaN
USGS_uv220443159235601 = 25 NaN
USGS_uv221101159280801 = 25 NaN
WUHH1 = 0 NaN
length of rain data: 25


# Calculations Normal Ratio Method for incomplete or 100% empty rain gages

It is important to NOTE:
-3 gages are required
-all 3 gages must have 100% pure raw rainfall data for best accuracy
-if any of the selected gages have missing rainfall data... the output of the function will produce nan. YOU MUST do the function however many times and then combine the best data

Example:
x1 = nan,2 ; x2=1,nan ; x3=2,3 ;  x4 = 1,2 x5 = nan, nan
if I choose x4, x2, x3 to find x5 : then the output of x5 will be 1, nan (1 was a random number)
I must then repeat the function doing
x4, x2, x3, to find x5 : then the output of x5 will be nan, 1 (1 was a random number)

THEN: i combine the two to get the best estimate of x5 : 1,1

.... HOPE this makes sense lol

In [21]:
Calculated_dict = {}

#(dataframe, dataframe, list, list)
def norm_ratio(gages_data, avg_rain, station_of_interest, select_gages):


    holding_dict = {}
    #some data formatting stuff.......
    normal_rain_C1 = avg_rain.loc[avg_rain['FILENAME'].isin(select_gages)]
    gages = normal_rain_C1['FILENAME'].tolist() ; rain = normal_rain_C1['RASTERVALU'].tolist()
    nr_C1_dict = {}
    for i in range(0, len(normal_rain_C1)):
        nr_C1_dict[gages[i]] = rain[i]


    est_rain = []
    Nc1 = avg_rain.RASTERVALU[avg_rain.index[avg_rain['FILENAME'] == station_of_interest][0]]    
    m=len(select_gages)
    for i in range(0, len(gages_data)):
        if np.isnan(gages_data[station_of_interest][i]):
            normalize = 0
            for y in range(0, len(gages)):
                normalize += (Nc1 / nr_C1_dict[gages[y]])*(gages_data[gages[y]][i])
            P = (1 / m) * (normalize)
            est_rain.append(P)
        else: 
            est_rain.append(gages_data[station_of_interest][i])
    holding_dict[station_of_interest] = est_rain

    return est_rain

In [22]:
#Nc1 = avg_rain.RASTERVALU[avg_rain.index[avg_rain['FILENAME'] == station_of_interest][0]]

In [23]:
#Nc1

In [27]:
#Choose the station of interest
station_of_interest = 'HI-45'
#Select which gages to include in the equation
select_gages = ['PRIH1', 'USC00518165', 'HI-41']

In [28]:
filled_na1 = norm_ratio(gages_data, avg_rain, station_of_interest, select_gages)
dfx = pd.DataFrame()
dfx['test'] = filled_na1
dfx.to_csv('test4.csv')

In [26]:
filled_na1.to_csv('HI-45.csv')

AttributeError: 'list' object has no attribute 'to_csv'

In [ ]:
#Choose the station of interest
station_of_interest = 'HI-45'
#Select which gages to include in the equation
select_gages = ['PR1HI' , 'USC00518165']

filled_na2 = norm_ratio(gages_data, avg_rain, station_of_interest, select_gages)

In [ ]:
HI_45 = []
for i in range(len(filled_na1)):
    if np.isnan(filled_na1[i]):
        x = filled_na2[i]
    else:
        x = filled_na1[i]
    HI_45.append(x)
        

In [ ]:
#Choose the station of interest
station_of_interest = 'USGS_uv221101159280801'
#Select which gages to include in the equation
select_gages = ['HI-45' , 'PR1HI' , 'USC00518165', 'USGS_uv220427159300201', 'KPIH1']

In [ ]:
mid_stream1 = norm_ratio(gages_data, avg_rain, station_of_interest, select_gages)

In [ ]:
#Choose the station of interest
station_of_interest = 'USGS_uv221101159280801'
#Select which gages to include in the equation
select_gages = ['PR1HI' , 'USC00518165', 'USGS_uv220427159300201', 'KPIH1' ]

In [ ]:
mid_stream2 = norm_ratio(gages_data, avg_rain, station_of_interest, select_gages)

In [ ]:
USGS_uv221101159280801 = []
for i in range(len(mid_stream1)):
    if np.isnan(mid_stream1[i]):
        x = mid_stream2[i]
    else:
        x = mid_stream1[i]
    USGS_uv221101159280801.append(x)

# If you used 100% filled rain gages to calculate the data proceed forward with this code... IF NOT: go into excel and glue the calculated values together 

In [ ]:
# for not excel....

In [ ]:
gages_data['HI-45'] = HI_45
gages_data['USGS_uv221101159280801'] = USGS_uv221101159280801

# Plot all of the data

In [ ]:
gages_names = (gages_data.columns.tolist())[2:]

fig, axs= plt.subplots(len(gages_names), figsize=(10,30))
fig.tight_layout(pad=3.0)

for i in range(0, len(gages_names)):
    axs[i].plot(gages_data['DateTime'], gages_data[gages_names[i]])
    axs[i].set_title(gages_names[i])
    axs[i].set_xlim((gages_data['DateTime'].min()), (gages_data['DateTime'].max()))

# Export Data

In [ ]:
gages_data.to_csv(all_rain_gages_data[:-4] + 'filled_NA.csv')